In [78]:
import os
import gzip
import uuid

import pandas as pd

from tqdm import tqdm
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
es = Elasticsearch()

data_dir = "clean/"

In [4]:
example = {
    'author': 'kimchy',
    'text': 'Elasticsearch: cool. bonsai cool.',
    'timestamp': datetime.now(),
}
res = es.index(index="test-index", id=1, body=example)
print(res['result'])

updated


In [ ]:
def read_one_day(day, directory):
    dir_names = [f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))]
    
    filenames = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    frames = []
    for f_name in filenames:
        if day in f_name:
            file = gzip.open(f"{directory}/{f_name}", "r")
            frames.append(pd.read_csv(file, sep=";"))
    
    df = pd.concat(frames)
    df.Timestamp = pd.to_datetime(df.Timestamp)
    df.index = df.Timestamp
    df.drop(["Timestamp"], inplace=True, axis=1)
    
    # Problèmes avec les données "clean": plusieurs jours d'enregistrements sont contenus dans les fichiers
    return df[df.index.day == datetime.datetime.strptime(day, '%Y-%m-%d').day].sort_index()

In [28]:
def load_data(directory):
    dir_names = [f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))]
    
    dfs = {}
    for d_name in tqdm(dir_names, "Directory"):
        filenames = [f for f in os.listdir(f"{directory}/{d_name}") if os.path.isfile(os.path.join(f"{directory}/{d_name}", f))]
        station = {}
        for f_name in filenames:
            if station.get(f_name[:10]) is None:
                station[f_name[:10]] = []
                
            file = gzip.open(f"{directory}/{d_name}/{f_name}", "r")
            station[f_name[:10]].append(pd.read_csv(file, sep=";"))
        
        station = {date: pd.concat(dfs).to_dict() for date, dfs in station.items()}
        dfs[d_name] = station
        
    return dfs

In [69]:
def load_all(data_dir=data_dir):
    dfs = []
    for d_name in tqdm(os.listdir("./clean/")):
        for f_name in os.listdir('./clean/' + d_name):
            file = gzip.open(os.path.join(data_dir, d_name, f_name), "r")
            dfs.append(pd.read_csv(file, sep=";"))

    return pd.concat(dfs)

In [99]:
dfs = load_all()



  0%|          | 0/25 [00:00<?, ?it/s]

  4%|▍         | 1/25 [00:09<03:40,  9.18s/it]

  8%|▊         | 2/25 [00:17<03:26,  8.96s/it]

 12%|█▏        | 3/25 [00:17<02:18,  6.30s/it]

 16%|█▌        | 4/25 [00:26<02:25,  6.91s/it]

 20%|██        | 5/25 [00:28<01:54,  5.71s/it]

 24%|██▍       | 6/25 [00:36<02:01,  6.41s/it]

 28%|██▊       | 7/25 [00:45<02:05,  6.95s/it]

 32%|███▏      | 8/25 [00:53<02:06,  7.43s/it]

 36%|███▌      | 9/25 [01:01<02:02,  7.64s/it]

 40%|████      | 10/25 [01:10<01:57,  7.80s/it]

 44%|████▍     | 11/25 [01:18<01:50,  7.90s/it]

 48%|████▊     | 12/25 [01:25<01:42,  7.86s/it]

 52%|█████▏    | 13/25 [01:34<01:38,  8.17s/it]

 56%|█████▌    | 14/25 [01:42<01:29,  8.15s/it]

 60%|██████    | 15/25 [01:50<01:20,  8.06s/it]

KeyboardInterrupt: 

In [97]:
dfs = dfs[(dfs["Timestamp"] > "2016-01-01 00:00:00") & (dfs["Timestamp"] < "2016-12-31 23:59:59")]
dfs.columns = [c.lower() for c in dfs.columns]
dfs.head()

,timestamp,station,bikes,slots,total,status,humidity,pressure,rain,winddeg,windspeed,snow,temperaturetemp
0,2016-09-22 09:00:00,01. Duc,5,4,9,clouds,26.0,1021.10,{},233.503,1.21,{},16.57
1,2016-09-22 09:10:00,01. Duc,5,4,9,clouds,92.0,1020.00,{},233.503,1.21,{},15.99
0,2016-04-06 02:50:00,01. Duc,2,8,10,clear,91.0,943.82,{},42.0004,1.06,{},7.60
1,2016-04-06 03:00:00,01. Duc,2,8,10,clear,91.0,943.82,{},42.0004,1.06,{},7.60
2,2016-04-06 03:10:00,01. Duc,2,8,10,clear,91.0,943.82,{},42.0004,1.06,{},7.60


In [98]:
len(dfs)

987419

In [94]:
for i, row in zip(range(1), dfs.iterrows()):
    print(row[1].WindDeg)

292.501


In [96]:
"""
id = 1
for row in dfs.iterrows():
    for date, df in station_dic.items():
        res = es.index(index=f"velos", id=id, body=df)
        print(res['result'])
        id += 1
"""

def gendata(dfs):
    for row in dfs.iterrows():
        for key, item in row[1].to_dict().items():
            if item == "None":
                row[1][key] = 0
                
        yield {
            "_id": row[0],
            "_index": "velos",
            "_type": "_doc",
            "_source": row[1].to_dict(),
        }

bulk(es, gendata(dfs))

KeyboardInterrupt: 